<a href="https://colab.research.google.com/github/sumaaithal/DeepLearning/blob/main/Google_stock_price_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

## preprocessing

In [2]:
train_df = pd.read_csv("/content/Google_Stock_Price_Train.csv")

In [4]:
training_set = train_df.iloc[:,1:2].values

In [7]:
#normalization
sc = MinMaxScaler(feature_range=(0,1))
scaled_training_set = sc.fit_transform(training_set)
scaled_training_set[:3]

array([[0.08581368],
       [0.09701243],
       [0.09433366]])

In [8]:
##creating 60 times steps(60 past steps) and 1 output
X_train = []
y_train = []

for i in range(60,1258):
  X_train.append(scaled_training_set[i-60:i,0])
  y_train.append(scaled_training_set[i,0])

X_train,y_train = np.array(X_train),np.array(y_train)

In [9]:
X_train,y_train

(array([[0.08581368, 0.09701243, 0.09433366, ..., 0.07846566, 0.08034452,
         0.08497656],
        [0.09701243, 0.09433366, 0.09156187, ..., 0.08034452, 0.08497656,
         0.08627874],
        [0.09433366, 0.09156187, 0.07984225, ..., 0.08497656, 0.08627874,
         0.08471612],
        ...,
        [0.92106928, 0.92438053, 0.93048218, ..., 0.95475854, 0.95204256,
         0.95163331],
        [0.92438053, 0.93048218, 0.9299055 , ..., 0.95204256, 0.95163331,
         0.95725128],
        [0.93048218, 0.9299055 , 0.93113327, ..., 0.95163331, 0.95725128,
         0.93796041]]),
 array([0.08627874, 0.08471612, 0.07454052, ..., 0.95725128, 0.93796041,
        0.93688146]))

In [10]:
## add new dimention
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

## building RNN

In [ ]:
## note : drop out regularization to avoid overfitting

In [13]:
#initialize the rnn
regressor = Sequential()

In [16]:
## adding our first lstm model
regressor.add( LSTM( units=50, return_sequences= True, input_shape=(X_train.shape[1],1) ) )
regressor.add( Dropout(rate=0.2) )

In [17]:
## add 2nd lstm layer
regressor.add( LSTM( units=50, return_sequences= True) )
regressor.add( Dropout(rate=0.2) )

In [18]:
## add 3rd lstm layer
regressor.add( LSTM( units=50, return_sequences= True) )
regressor.add( Dropout(rate=0.2) )

In [19]:
## add 4th lstm layer
regressor.add( LSTM( units=50, return_sequences= False) )
regressor.add( Dropout(rate=0.2) )

In [20]:
## add output layer
regressor.add( Dense(units=1) )

In [22]:
## compiling the RNN
regressor.compile(optimizer='adam',loss='mean_squared_error')